# Testing for E2 - using all spindles, training with all data
fistly define the UUID of the experiment to use

In [172]:
experimentId="56c86206-b216-4e8a-b34a-03d33136ac83"

-------------------------------------------------------

In [173]:
#external libraries
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.colors as clt
import plotly
import plotly.subplots as sb
import plotly.express as px
import plotly.graph_objects as go
import dotenv
import pandas as pd
import scipy.fft as fft
import scipy.signal as sg
import scipy.io as sio
import pickle as pkl
import xgboost as xgb
import time
import sklearn.metrics as skm

#project library
from spinco import *

#environment variables
dotenv.load_dotenv('lab.env')

#project variables
datapath=os.environ['DATAPATH']
cognipath=datapath+"\\COGNITION"
dreamspath=datapath+"\\DREAMS"
masspath=datapath+"\\MASS"


## load mass

In [174]:
samplerate=200  #Should rethink this

In [175]:
annotations, signalsMetadata = loadMASSSpindles(masspath,forceSamplerate=samplerate)

In [176]:
#consider to include this in a function
minDuration=0.3
maxDuration=5
annotations=annotations[annotations.type=='spindle']
annotations=annotations[annotations.duration>minDuration]
annotations=annotations[annotations.duration<maxDuration]
annotations=annotations.reset_index(drop=True)

In [177]:
#check with EDA results (not needed)
print(1-len(annotations)/33458)

0.005648873214178951


In [178]:
annotations.head()

,type,expert,subjectId,labelerId,startTime,duration,samplerate,stopTime,startInd,stopInd
0,spindle,E1,0001,0001,888.327805,0.640579,200,888.968384,177666,177794
1,spindle,E1,0001,0001,905.758061,0.578094,200,906.336155,181152,181267
2,spindle,E1,0001,0001,917.731574,0.847603,200,918.579177,183546,183716
3,spindle,E1,0001,0001,922.078189,0.878845,200,922.957034,184416,184591
4,spindle,E1,0001,0001,939.055445,0.757767,200,939.813212,187811,187963


In [179]:
signalsMetadata.head()

,subjectId,file,channel,duration,samplerate,isOriginalSamplerate,database
0,0001,MASS_0001.pkl,C3-CLE,28956.0,200,False,MASS
1,0002,MASS_0002.pkl,C3-CLE,35016.0,200,False,MASS
2,0003,MASS_0003.pkl,C3-CLE,36760.0,200,False,MASS
3,0004,MASS_0004.pkl,C3-CLE,28004.0,200,False,MASS
4,0005,MASS_0005.pkl,C3-CLE,31244.0,200,False,MASS


## Load experiment results

In [180]:
experimentModels, featureSelection = loadExperiment(experimentId,datapath)

In [181]:
experimentModels

,criteriumId,criteriumName,labelerIdList,train,val,test,modelId,spindleTimeRate
0,0001,E1,[0001],"[0003, 0005, 0007, 0009, 0010, 0011, 0012, 001...","[0017, 0002, 0006]",0001,bdbdfc7b-0c97-484c-b4e9-70e698d9b353,0.018639
1,0001,E1,[0001],"[0001, 0003, 0005, 0007, 0009, 0010, 0011, 001...","[0006, 0013, 0019]",0002,bae09da0-598a-46ed-b46a-553a9d1de380,0.020055
2,0001,E1,[0001],"[0002, 0005, 0007, 0009, 0010, 0011, 0012, 001...","[0013, 0006, 0001]",0003,2c77d9d9-8a36-4e76-ac92-65f9318bd100,0.021075
3,0001,E1,[0001],"[0001, 0002, 0003, 0006, 0007, 0009, 0010, 001...","[0018, 0014, 0012]",0005,dc49a81b-fa55-4b8e-989b-e40bd13a4302,0.018331
4,0001,E1,[0001],"[0003, 0007, 0009, 0010, 0011, 0012, 0013, 001...","[0005, 0001, 0002]",0006,3d3c90d6-8d87-4dd0-85ec-9eedee1565cb,0.019222
5,0001,E1,[0001],"[0001, 0002, 0003, 0010, 0011, 0012, 0013, 001...","[0005, 0006, 0009]",0007,6622a394-926c-45ad-a0f5-eb1def948317,0.019625
6,0001,E1,[0001],"[0001, 0003, 0005, 0006, 0007, 0010, 0011, 001...","[0002, 0017, 0013]",0009,91153884-8b2d-49a1-86cd-386a1add8f8e,0.017824
7,0001,E1,[0001],"[0002, 0003, 0005, 0006, 0007, 0009, 0012, 001...","[0017, 0001, 0011]",0010,672f9a5b-bab6-4a47-8889-b20041f97fe8,0.017542
8,0001,E1,[0001],"[0001, 0003, 0005, 0006, 0009, 0010, 0012, 001...","[0002, 0018, 0007]",0011,98885354-4ef8-4d7f-b64d-8fb9a1edc578,0.015442
9,0001,E1,[0001],"[0002, 0003, 0005, 0006, 0009, 0011, 0013, 001...","[0001, 0007, 0010]",0012,39389298-e7f5-4de1-9dc7-bcf9a95245bd,0.016460


In [182]:
#we show the difference in class inbalance for the annotation criteria considered
experimentModels[['criteriumName','spindleTimeRate']].groupby('criteriumName').describe()

spindleTimeRate                                          \
                        count      mean       std       min       25%   
criteriumName                                                           
E1                       15.0  0.018463  0.001834  0.015442  0.017419   
E2                       15.0  0.058320  0.004520  0.052347  0.054965   
union                    15.0  0.059557  0.004579  0.053738  0.055972   

                                             
                    50%       75%       max  
criteriumName                                
E1             0.018331  0.019424  0.022439  
E2             0.056746  0.061623  0.067005  
union          0.058222  0.062711  0.068526

In [183]:
featureSelection

,window,characteristic,bandName,Times_selected
0,2,sigmaIndex,broadband,1000.0
1,2,hjortActivity,sigma,1000.0
2,1.5,sigmaIndex,broadband,1000.0
3,1,hjortActivity,sigma,1000.0
4,1,sigmaIndex,broadband,1000.0
5,2,hjortActivity,beta1,1000.0
6,2,hjortActivity,beta2,1000.0
7,1.5,hjortActivity,sigma,999.0
8,2,hjortMobility,beta1,994.0
9,2,hjortActivity,delta2,986.0


## Hyperparameter definition
this should come from a previous evaluation notebook

In [184]:
auxThres=pd.DataFrame({
    'hyperThres':[0.1, 0.2,0.3, 0.4, 0.5]
})
auxDepth=pd.DataFrame({
    'hyperDepth':[10,20,30,40, 50, 60]
})
auxMinDuration=pd.DataFrame({
    'hyperMinDuration':[0.3]
})
auxMaxDuration=pd.DataFrame({
    'hyperMaxDuration':[5]
})
auxClose=pd.DataFrame({
    'hyperClose':[0.1]
})


hyperParams=pd.merge(auxThres,auxDepth,how='cross')
hyperParams=pd.merge(hyperParams,auxMinDuration,how='cross')
hyperParams=pd.merge(hyperParams,auxMaxDuration,how='cross')
hyperParams=pd.merge(hyperParams,auxClose,how='cross')

hyperParams

,hyperThres,hyperDepth,hyperMinDuration,hyperMaxDuration,hyperClose
0,0.1,10,0.3,5,0.1
1,0.1,20,0.3,5,0.1
2,0.1,30,0.3,5,0.1
3,0.1,40,0.3,5,0.1
4,0.1,50,0.3,5,0.1
5,0.1,60,0.3,5,0.1
6,0.2,10,0.3,5,0.1
7,0.2,20,0.3,5,0.1
8,0.2,30,0.3,5,0.1
9,0.2,40,0.3,5,0.1


## Testing with E2 criterium
we test the optimal points for the prediction threshold and number of boost iterations in the different validation groups

In [185]:
experimentModels=experimentModels[experimentModels.criteriumName=='E2'].reset_index(drop=True)
experimentModels

,criteriumId,criteriumName,labelerIdList,train,val,test,modelId,spindleTimeRate
0,0002,E2,[0002],"[0003, 0005, 0007, 0009, 0010, 0011, 0012, 001...","[0017, 0002, 0006]",0001,27775e1b-9adc-4ae7-9fcf-b3dbe38ca573,0.056746
1,0002,E2,[0002],"[0001, 0003, 0005, 0007, 0009, 0010, 0011, 001...","[0006, 0013, 0019]",0002,363d6b1b-23e4-4aa5-8a2e-a2aa6c9a2bd0,0.061232
2,0002,E2,[0002],"[0002, 0005, 0007, 0009, 0010, 0011, 0012, 001...","[0013, 0006, 0001]",0003,09d246cf-df61-4da1-b690-e0c8338d819c,0.063062
3,0002,E2,[0002],"[0001, 0002, 0003, 0006, 0007, 0009, 0010, 001...","[0018, 0014, 0012]",0005,d7547448-90e9-4379-8b6f-40fc4ce250cf,0.061716
4,0002,E2,[0002],"[0003, 0007, 0009, 0010, 0011, 0012, 0013, 001...","[0005, 0001, 0002]",0006,8494b8dc-14c3-4df3-a9e2-2569fced6b66,0.058705
5,0002,E2,[0002],"[0001, 0002, 0003, 0010, 0011, 0012, 0013, 001...","[0005, 0006, 0009]",0007,4ec9c4e1-e050-4cb3-911d-426930925d09,0.061530
6,0002,E2,[0002],"[0001, 0003, 0005, 0006, 0007, 0010, 0011, 001...","[0002, 0017, 0013]",0009,9f43f1cd-280c-4e7c-9c3a-a90f704900c2,0.055908
7,0002,E2,[0002],"[0002, 0003, 0005, 0006, 0007, 0009, 0012, 001...","[0017, 0001, 0011]",0010,285a2ff5-4005-4d2b-a89a-5108d2058c0f,0.052347
8,0002,E2,[0002],"[0001, 0003, 0005, 0006, 0009, 0010, 0012, 001...","[0002, 0018, 0007]",0011,7bee7b22-c6a4-482c-866e-cfd36aa7b2f8,0.054834
9,0002,E2,[0002],"[0002, 0003, 0005, 0006, 0009, 0011, 0013, 001...","[0001, 0007, 0010]",0012,883a5eb1-e984-44b0-91e9-1c97b07d1547,0.053001


In [186]:
experimentModels_N2 = experimentModels.copy()
experimentModels_N2.head()

,criteriumId,criteriumName,labelerIdList,train,val,test,modelId,spindleTimeRate
0,0002,E2,[0002],"[0003, 0005, 0007, 0009, 0010, 0011, 0012, 001...","[0017, 0002, 0006]",0001,27775e1b-9adc-4ae7-9fcf-b3dbe38ca573,0.056746
1,0002,E2,[0002],"[0001, 0003, 0005, 0007, 0009, 0010, 0011, 001...","[0006, 0013, 0019]",0002,363d6b1b-23e4-4aa5-8a2e-a2aa6c9a2bd0,0.061232
2,0002,E2,[0002],"[0002, 0005, 0007, 0009, 0010, 0011, 0012, 001...","[0013, 0006, 0001]",0003,09d246cf-df61-4da1-b690-e0c8338d819c,0.063062
3,0002,E2,[0002],"[0001, 0002, 0003, 0006, 0007, 0009, 0010, 001...","[0018, 0014, 0012]",0005,d7547448-90e9-4379-8b6f-40fc4ce250cf,0.061716
4,0002,E2,[0002],"[0003, 0007, 0009, 0010, 0011, 0012, 0013, 001...","[0005, 0001, 0002]",0006,8494b8dc-14c3-4df3-a9e2-2569fced6b66,0.058705


In [187]:
annotationsN2=pd.read_csv(datapath+'/MASS/annotations/spindlesFilteredN2.csv')
annotationsN2['samplerate']=samplerate
annotationsN2['subjectId']=annotationsN2.apply(
    lambda row: str(row.subjectId).zfill(4),axis=1)
annotationsN2['labelerId']=annotationsN2.apply(
    lambda row: str(row.labelerId).zfill(4),axis=1)
annotationsN2['stopTime']=annotationsN2.apply(
    lambda row: row.startTime+row.duration , axis=1)
annotationsN2['startInd']=annotationsN2.apply(
    lambda row: seconds2index(row.startTime,row.samplerate) , axis=1)
annotationsN2['stopInd']=annotationsN2.apply(
    lambda row: seconds2index(row.stopTime,row.samplerate) , axis=1)

In [188]:
## load hypnograms
stagesAnnotations=pd.read_csv(datapath+"\MASS\stages\stages.csv")
stagesAnnotations['samplerate']=samplerate
stagesAnnotations['subjectId']=stagesAnnotations.apply(
    lambda row: str(row.subjectId).zfill(4),axis=1)
stagesAnnotations['stopTime']=stagesAnnotations.apply(
    lambda row: row.startTime+row.duration , axis=1)
stagesAnnotations['startInd']=stagesAnnotations.apply(
    lambda row: seconds2index(row.startTime,row.samplerate) , axis=1)
stagesAnnotations['stopInd']=stagesAnnotations.apply(
    lambda row: seconds2index(row.stopTime,row.samplerate) , axis=1)
stagesAnnotations.head(5)

hypnograms={}
for ind, row in signalsMetadata.iterrows():
    subjectId=row.subjectId
    thisStages=stagesAnnotations[stagesAnnotations.subjectId==subjectId]
    excerptDimension=int(row.duration*row.samplerate)
    thisHypnogram=np.ones((excerptDimension,))*np.nan
    for ind_stg, row_stg in thisStages.iterrows():
        thisHypnogram[row_stg.startInd:row_stg.stopInd]=row_stg.value
    hypnograms[subjectId]=thisHypnogram  

In [189]:
rawF1s = []
rawPrecisions = []
rawRecalls = []

rawThres = []
rawDepth = []

f1s = []
precisions = []
recalls = []

eventF1s = []
eventPrecisions = []
eventRecalls = []

checks = []

rawF1s_N2 = []
rawPrecisions_N2 = []
rawRecalls_N2 = []

f1s_N2 = []
precisions_N2 = []
recalls_N2 = []

eventF1s_N2 = []
eventPrecisions_N2 = []
eventRecalls_N2 = []

checks_N2 = []

thisExperimentModels = experimentModels.copy()
for ind, row in thisExperimentModels.iterrows():
    #load model
    print(ind)
    model = loadBooster(row.modelId, experimentId, datapath)
    #initialise lists
    rawF1xs=[]

    

    #iterate validation subjects
    for valSubjectId in row.val:
        print(valSubjectId)
        #Define annotations criterium
        usedAnnotations=annotations[annotations.labelerId.isin(row.labelerIdList)].reset_index(drop=True)
        #Load features and labels
        valFeatures=loadFeatureMatrix([valSubjectId],featureSelection,signalsMetadata,samplerate,datapath)
        valLabels=loadLabelsVector([valSubjectId],usedAnnotations,signalsMetadata,samplerate)
        #Predict
        valDMatrix=xgb.DMatrix(data=valFeatures)
        probabilities=[]
        for i_depth, j_depth in auxDepth.iterrows():
            pred=model.predict(valDMatrix,iteration_range=(0,int(j_depth['hyperDepth'])))
            probabilities.append(pred)
        probabilities=np.array(probabilities)
        f1xs=[]

        
        for ind_hyper,row_hyper in hyperParams.iterrows():
            thisExperimentModels=experimentModels.copy()
            hyperThres=row_hyper.hyperThres
            hyperDepth=int(row_hyper.hyperDepth)
            hyperMinDuration=row_hyper.hyperMinDuration
            hyperMaxDuration=row_hyper.hyperMaxDuration
            hyperClose=row_hyper.hyperClose
            probs=probabilities[auxDepth['hyperDepth']==hyperDepth][0]
            raw=probs>=hyperThres
            #Processed labels
            processed=labelingProcess(raw,hyperClose,hyperMinDuration,samplerate)
            gtAnnotations=labelVectorToAnnotations(valLabels,samplerate)
            detections=labelVectorToAnnotations(processed,samplerate)
            detections=detections[detections.duration<hyperMaxDuration]
            #Metrics
            f,r,p=annotationPairToMetrics(gtAnnotations,detections, thresIoU=0.2)
        
            #Metric appends
            f1xs.append(f)

        rawF1xs.append(np.array(f1xs))

    #statistics of the metrics over the subjects of the validation set
    meanF1s=np.mean(np.array(rawF1xs), axis=0)
    
    optimalInd=hyperParams[meanF1s==np.max(meanF1s)].iloc[0]
    hyperParams_opt=optimalInd
    print(hyperParams_opt)
    hyperThres=hyperParams_opt['hyperThres']
    hyperDepth=int(hyperParams_opt['hyperDepth'])
    hyperMinDuration=hyperParams_opt['hyperMinDuration']
    hyperMaxDuration=hyperParams_opt['hyperMaxDuration']
    hyperClose=hyperParams_opt['hyperClose']
    
    ##
    rawThres.append(hyperThres)
    rawDepth.append(hyperDepth)


    testSubjectId=row.test
    #Define annotations criterium
    usedAnnotations=annotations[annotations.labelerId.isin(row.labelerIdList)].reset_index(drop=True)
    #Load features and labels
    testFeatures=loadFeatureMatrix([testSubjectId],featureSelection,signalsMetadata,samplerate,datapath)
    testLabels=loadLabelsVector([testSubjectId],usedAnnotations,signalsMetadata,samplerate)

    #Predict
    testDMatrix=xgb.DMatrix(data=testFeatures)
    probabilities=model.predict(testDMatrix,iteration_range=(0,hyperDepth))
    rawLabels=probabilities>=hyperThres
    #Raw Metrics
    rawTp=np.sum(rawLabels*testLabels)
    rawFp=np.sum(rawLabels*(1-testLabels))
    rawTn=np.sum((1-rawLabels)*(1-testLabels))
    rawFn=np.sum((1-rawLabels)*testLabels)
    #Raw appends
    rawF1s.append(2*rawTp/(2*rawTp+rawFp+rawFn))
    rawPrecisions.append(rawTp/(rawTp+rawFp) )
    rawRecalls.append(rawTp/(rawTp+rawFn))
    #Process
    processedLabels=labelingProcess(rawLabels,hyperClose,hyperMinDuration,samplerate)
    #Processed metrics
    tp=np.sum(processedLabels*testLabels)
    fp=np.sum(processedLabels*(1-testLabels))
    tn=np.sum((1-processedLabels)*(1-testLabels))
    fn=np.sum((1-processedLabels)*testLabels)
    #Processed appends
    f1s.append(2*tp/(2*tp+fp+fn))
    precisions.append(tp/(tp+fp))
    recalls.append(tp/(tp+fn))

    #By-event metrics
    processedAnnotations=labelVectorToAnnotations(processedLabels,samplerate)
    processedAnnotations=processedAnnotations[processedAnnotations.duration<maxDuration]
    gtAnnotations=labelVectorToAnnotations(testLabels,samplerate)   #<- or just filter the annotations
    f,r,p=annotationPairToMetrics(gtAnnotations,processedAnnotations, thresIoU=0.2)
    print('F1-score')
    print(f)
    #calculate metrics
    eventF1s.append(f)
    eventPrecisions.append(p)
    eventRecalls.append(r)
    
    #N2
    testHypnogram=hypnograms[testSubjectId]
    testN2Mask=testHypnogram==2
    #Define annotations criterium
    usedAnnotations=annotationsN2[annotationsN2.labelerId.isin(row.labelerIdList)].reset_index(drop=True)
    #Load labels
    testLabels=loadLabelsVector([testSubjectId],usedAnnotations,signalsMetadata,samplerate)

    #Apply mask to predictions---------------------------------->
    rawLabels=rawLabels*testN2Mask

    
    #Raw Metrics
    rawTp = np.sum(rawLabels * testLabels)
    rawFp = np.sum(rawLabels * (1 - testLabels))
    rawTn = np.sum((1 - rawLabels) * (1 - testLabels))
    rawFn = np.sum((1 - rawLabels) * testLabels)
    #Raw appends
    rawF1s_N2.append(2 * rawTp / (2 * rawTp + rawFp + rawFn))
    rawPrecisions_N2.append(rawTp / (rawTp + rawFp))
    rawRecalls_N2.append(rawTp / (rawTp + rawFn))
    #Process
    processedLabels=labelingProcess(rawLabels,hyperClose,hyperMinDuration,samplerate)
    #Processed metrics
    tp = np.sum(processedLabels * testLabels)
    fp = np.sum(processedLabels * (1 - testLabels))
    tn = np.sum((1 - processedLabels) * (1 - testLabels))
    fn = np.sum((1 - processedLabels) * testLabels)
    #Processed appends
    f1s_N2.append(2 * tp / (2 * tp + fp + fn))
    precisions_N2.append(tp / (tp + fp))
    recalls_N2.append(tp / (tp + fn))

    #By-event metrics
    processedAnnotations = labelVectorToAnnotations(processedLabels, samplerate)
    gtAnnotations = labelVectorToAnnotations(testLabels, samplerate)  #<- or just filter the annotations
    f, r, p = annotationPairToMetrics(gtAnnotations, processedAnnotations, thresIoU=0.2)
    print('N2 F1-score')
    print(f)
    #calculate metrics
    eventF1s_N2.append(f)
    eventPrecisions_N2.append(p)
    eventRecalls_N2.append(r)

#include metrics in the dataframe
experimentModels['rawF1'] = rawF1s
experimentModels['rawPrecision'] = rawPrecisions
experimentModels['rawRecall'] = rawRecalls

experimentModels['f1'] = f1s
experimentModels['precision'] = precisions
experimentModels['recall'] = recalls

experimentModels['eventF1'] = eventF1s
experimentModels['eventPrecision'] = eventPrecisions
experimentModels['eventRecall'] = eventRecalls

experimentModels['Binarization_Threshold'] = rawF1s
experimentModels['Boosting_Iterations'] = rawPrecisions
#include metrics in the N2 dataframe
experimentModels_N2['rawF1'] = rawF1s_N2
experimentModels_N2['rawPrecision'] = rawPrecisions_N2
experimentModels_N2['rawRecall'] = rawRecalls_N2

experimentModels_N2['f1'] = f1s_N2
experimentModels_N2['precision'] = precisions_N2
experimentModels_N2['recall'] = recalls_N2

experimentModels_N2['eventF1'] = eventF1s_N2
experimentModels_N2['eventPrecision'] = eventPrecisions_N2
experimentModels_N2['eventRecall'] = eventRecalls_N2

experimentModels_N2['Binarization_Threshold'] = rawF1s
experimentModels_N2['Boosting_Iterations'] = rawPrecisions


0
0017
0002
0006
hyperThres           0.1
hyperDepth          40.0
hyperMinDuration     0.3
hyperMaxDuration     5.0
hyperClose           0.1
Name: 3, dtype: float64
F1-score
0.8034670429348921
N2 F1-score
0.8846584546472565
1
0006
0013
0019
hyperThres           0.3
hyperDepth          40.0
hyperMinDuration     0.3
hyperMaxDuration     5.0
hyperClose           0.1
Name: 15, dtype: float64
F1-score
0.8587462939432444
N2 F1-score
0.8723547951373255
2
0013
0006
0001
hyperThres           0.2
hyperDepth          60.0
hyperMinDuration     0.3
hyperMaxDuration     5.0
hyperClose           0.1
Name: 11, dtype: float64
F1-score
0.6181719848566792
N2 F1-score
0.7576361221779548
3
0018
0014
0012
hyperThres           0.5
hyperDepth          10.0
hyperMinDuration     0.3
hyperMaxDuration     5.0
hyperClose           0.1
Name: 24, dtype: float64
F1-score
0.776757005052825
N2 F1-score
0.7918424753867792
4
0005
0001
0002
hyperThres           0.2
hyperDepth          50.0
hyperMinDuration     0.3
hyperM

In [190]:
dumpPickle('experimentModels_ValidationperModel_Testing_E2_IoU-0.2_temp.pkl',experimentModels)

In [191]:
experimentModels.columns

Index(['criteriumId', 'criteriumName', 'labelerIdList', 'train', 'val', 'test',
       'modelId', 'spindleTimeRate', 'rawF1', 'rawPrecision', 'rawRecall',
       'f1', 'precision', 'recall', 'eventF1', 'eventPrecision', 'eventRecall',
       'Binarization_Threshold', 'Boosting_Iterations'],
      dtype='object')

In [192]:
fig=px.scatter(experimentModels,x='rawF1',y='f1',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['rawF1'], y=experimentModels['rawF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [193]:
fig=px.scatter(experimentModels,x='rawF1',y='eventF1',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['rawF1'], y=experimentModels['rawF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [194]:
fig=px.scatter(experimentModels,x='eventF1',y='eventPrecision',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['eventF1'], y=experimentModels['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [195]:
fig=px.scatter(experimentModels,x='eventF1',y='eventRecall',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['eventF1'], y=experimentModels['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [196]:
experimentModels[['test','eventF1','eventPrecision','eventRecall']].groupby('test').describe(percentiles=[0.5])

eventF1                                             eventPrecision  \
       count      mean std       min       50%       max          count   
test                                                                      
0001     1.0  0.803467 NaN  0.803467  0.803467  0.803467            1.0   
0002     1.0  0.858746 NaN  0.858746  0.858746  0.858746            1.0   
0003     1.0  0.618172 NaN  0.618172  0.618172  0.618172            1.0   
0005     1.0  0.776757 NaN  0.776757  0.776757  0.776757            1.0   
0006     1.0  0.625557 NaN  0.625557  0.625557  0.625557            1.0   
0007     1.0  0.697388 NaN  0.697388  0.697388  0.697388            1.0   
0009     1.0  0.827692 NaN  0.827692  0.827692  0.827692            1.0   
0010     1.0  0.791476 NaN  0.791476  0.791476  0.791476            1.0   
0011     1.0  0.831321 NaN  0.831321  0.831321  0.831321            1.0   
0012     1.0  0.715254 NaN  0.715254  0.715254  0.715254            1.0   
0013     1.0  0.780095 NaN  0.780095  0.780095  0.780095            1.0   
0014     1.0  0.773679 NaN  0.773679  0.773679  0.773679            1.0   
0017     1.0  0.758772 NaN  0.758772  0.758772  0.758772            1.0   
0018     1.0  0.794176 NaN  0.794176  0.794176  0.794176            1.0   
0019     1.0  0.774101 NaN  0.774101  0.774101  0.774101            1.0   

                                                 eventRecall                \
          mean std       min       50%       max       count      mean std   
test                                                                         
0001  0.778481 NaN  0.778481  0.778481  0.778481         1.0  0.829429 NaN   
0002  0.802220 NaN  0.802220  0.802220  0.802220         1.0  0.923602 NaN   
0003  0.454545 NaN  0.454545  0.454545  0.454545         1.0  0.963025 NaN   
0005  0.850101 NaN  0.850101  0.850101  0.850101         1.0  0.715131 NaN   
0006  0.803059 NaN  0.803059  0.803059  0.803059         1.0  0.512758 NaN   
0007  0.574770 NaN  0.574770  0.574770  0.574770         1.0  0.882242 NaN   
0009  0.835309 NaN  0.835309  0.835309  0.835309         1.0  0.820279 NaN   
0010  0.735519 NaN  0.735519  0.735519  0.735519         1.0  0.856179 NaN   
0011  0.800369 NaN  0.800369  0.800369  0.800369         1.0  0.864474 NaN   
0012  0.594121 NaN  0.594121  0.594121  0.594121         1.0  0.896698 NaN   
0013  0.675816 NaN  0.675816  0.675816  0.675816         1.0  0.921404 NaN   
0014  0.665915 NaN  0.665915  0.665915  0.665915         1.0  0.922693 NaN   
0017  0.791400 NaN  0.791400  0.791400  0.791400         1.0  0.728728 NaN   
0018  0.751082 NaN  0.751082  0.751082  0.751082         1.0  0.842296 NaN   
0019  0.675250 NaN  0.675250  0.675250  0.675250         1.0  0.905714 NaN   

                                    
           min       50%       max  
test                                
0001  0.829429  0.829429  0.829429  
0002  0.923602  0.923602  0.923602  
0003  0.963025  0.963025  0.963025  
0005  0.715131  0.715131  0.715131  
0006  0.512758  0.512758  0.512758  
0007  0.882242  0.882242  0.882242  
0009  0.820279  0.820279  0.820279  
0010  0.856179  0.856179  0.856179  
0011  0.864474  0.864474  0.864474  
0012  0.896698  0.896698  0.896698  
0013  0.921404  0.921404  0.921404  
0014  0.922693  0.922693  0.922693  
0017  0.728728  0.728728  0.728728  
0018  0.842296  0.842296  0.842296  
0019  0.905714  0.905714  0.905714

In [197]:
experimentModels[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean()

,test,eventF1,eventPrecision,eventRecall
0,0001,0.803467,0.778481,0.829429
1,0002,0.858746,0.802220,0.923602
2,0003,0.618172,0.454545,0.963025
3,0005,0.776757,0.850101,0.715131
4,0006,0.625557,0.803059,0.512758
5,0007,0.697388,0.574770,0.882242
6,0009,0.827692,0.835309,0.820279
7,0010,0.791476,0.735519,0.856179
8,0011,0.831321,0.800369,0.864474
9,0012,0.715254,0.594121,0.896698


In [198]:
experimentModels[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean().mean()

test              6.668000e+54
eventF1           7.617770e-01
eventPrecision    7.191971e-01
eventRecall       8.389767e-01
dtype: float64

In [199]:
experimentModels[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean().std()

C:\Users\GIB\AppData\Local\Temp\ipykernel_20560\2599554826.py:1: FutureWarning:

The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.



eventF1           0.070003
eventPrecision    0.111592
eventRecall       0.113804
dtype: float64

In [200]:
auxPrecision=pd.DataFrame({
    'metric':'event precision',
    'value':experimentModels.eventPrecision,
    'event F1':experimentModels.eventF1
})

auxRecall=pd.DataFrame({
    'metric':'event recall',
    'value':experimentModels.eventRecall,
    'event F1':experimentModels.eventF1
})
visualTradeoff=pd.concat((auxPrecision,auxRecall))

In [201]:
fig=px.scatter(visualTradeoff,x='event F1',y='value',color='metric', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['eventF1'], y=experimentModels['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

## Restrict to only N2 stage

In [202]:
dumpPickle('experimentModelsTest_ValidationperModel_Testing_N2_E2_IoU-0.2_temp.pkl',experimentModels_N2)

In [203]:
fig=px.scatter(experimentModels_N2,x='rawF1',y='f1',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels_N2['rawF1'], y=experimentModels_N2['rawF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [204]:
fig=px.scatter(experimentModels_N2,x='rawF1',y='eventF1',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels_N2['rawF1'], y=experimentModels_N2['rawF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [205]:
fig=px.scatter(experimentModels_N2,x='eventF1',y='eventPrecision',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels_N2['eventF1'], y=experimentModels_N2['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [206]:
fig=px.scatter(experimentModels_N2,x='eventF1',y='eventRecall',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels_N2['eventF1'], y=experimentModels_N2['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [207]:
experimentModels_N2[['test','eventF1','eventPrecision','eventRecall']].groupby('test').describe(percentiles=[0.5])

eventF1                                             eventPrecision  \
       count      mean std       min       50%       max          count   
test                                                                      
0001     1.0  0.884658 NaN  0.884658  0.884658  0.884658            1.0   
0002     1.0  0.872355 NaN  0.872355  0.872355  0.872355            1.0   
0003     1.0  0.757636 NaN  0.757636  0.757636  0.757636            1.0   
0005     1.0  0.791842 NaN  0.791842  0.791842  0.791842            1.0   
0006     1.0  0.655226 NaN  0.655226  0.655226  0.655226            1.0   
0007     1.0  0.809694 NaN  0.809694  0.809694  0.809694            1.0   
0009     1.0  0.842105 NaN  0.842105  0.842105  0.842105            1.0   
0010     1.0  0.890011 NaN  0.890011  0.890011  0.890011            1.0   
0011     1.0  0.842477 NaN  0.842477  0.842477  0.842477            1.0   
0012     1.0  0.834550 NaN  0.834550  0.834550  0.834550            1.0   
0013     1.0  0.815274 NaN  0.815274  0.815274  0.815274            1.0   
0014     1.0  0.797802 NaN  0.797802  0.797802  0.797802            1.0   
0017     1.0  0.804684 NaN  0.804684  0.804684  0.804684            1.0   
0018     1.0  0.814597 NaN  0.814597  0.814597  0.814597            1.0   
0019     1.0  0.770168 NaN  0.770168  0.770168  0.770168            1.0   

                                                 eventRecall                \
          mean std       min       50%       max       count      mean std   
test                                                                         
0001  0.949830 NaN  0.949830  0.949830  0.949830         1.0  0.829187 NaN   
0002  0.826849 NaN  0.826849  0.826849  0.826849         1.0  0.922967 NaN   
0003  0.623220 NaN  0.623220  0.623220  0.623220         1.0  0.964587 NaN   
0005  0.885624 NaN  0.885624  0.885624  0.885624         1.0  0.716102 NaN   
0006  0.906926 NaN  0.906926  0.906926  0.906926         1.0  0.513415 NaN   
0007  0.748197 NaN  0.748197  0.748197  0.748197         1.0  0.880769 NaN   
0009  0.867454 NaN  0.867454  0.867454  0.867454         1.0  0.818405 NaN   
0010  0.928041 NaN  0.928041  0.928041  0.928041         1.0  0.855201 NaN   
0011  0.820817 NaN  0.820817  0.820817  0.820817         1.0  0.865107 NaN   
0012  0.781250 NaN  0.781250  0.781250  0.781250         1.0  0.895147 NaN   
0013  0.732195 NaN  0.732195  0.732195  0.732195         1.0  0.918681 NaN   
0014  0.702375 NaN  0.702375  0.702375  0.702375         1.0  0.922638 NaN   
0017  0.897597 NaN  0.897597  0.897597  0.897597         1.0  0.729202 NaN   
0018  0.788692 NaN  0.788692  0.788692  0.788692         1.0  0.842137 NaN   
0019  0.669355 NaN  0.669355  0.669355  0.669355         1.0  0.905512 NaN   

                                    
           min       50%       max  
test                                
0001  0.829187  0.829187  0.829187  
0002  0.922967  0.922967  0.922967  
0003  0.964587  0.964587  0.964587  
0005  0.716102  0.716102  0.716102  
0006  0.513415  0.513415  0.513415  
0007  0.880769  0.880769  0.880769  
0009  0.818405  0.818405  0.818405  
0010  0.855201  0.855201  0.855201  
0011  0.865107  0.865107  0.865107  
0012  0.895147  0.895147  0.895147  
0013  0.918681  0.918681  0.918681  
0014  0.922638  0.922638  0.922638  
0017  0.729202  0.729202  0.729202  
0018  0.842137  0.842137  0.842137  
0019  0.905512  0.905512  0.905512

In [208]:
experimentModels_N2[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean()

,test,eventF1,eventPrecision,eventRecall
0,0001,0.884658,0.949830,0.829187
1,0002,0.872355,0.826849,0.922967
2,0003,0.757636,0.623220,0.964587
3,0005,0.791842,0.885624,0.716102
4,0006,0.655226,0.906926,0.513415
5,0007,0.809694,0.748197,0.880769
6,0009,0.842105,0.867454,0.818405
7,0010,0.890011,0.928041,0.855201
8,0011,0.842477,0.820817,0.865107
9,0012,0.834550,0.781250,0.895147


In [209]:
experimentModels_N2[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean().mean()

test              6.668000e+54
eventF1           8.122053e-01
eventPrecision    8.085616e-01
eventRecall       8.386038e-01
dtype: float64

In [210]:
experimentModels_N2[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean().std()

C:\Users\GIB\AppData\Local\Temp\ipykernel_20560\292585311.py:1: FutureWarning:

The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.



eventF1           0.058137
eventPrecision    0.099048
eventRecall       0.113429
dtype: float64

In [211]:
auxPrecision=pd.DataFrame({
    'metric':'event precision',
    'value':experimentModels_N2.eventPrecision,
    'event F1':experimentModels_N2.eventF1
})

auxRecall=pd.DataFrame({
    'metric':'event recall',
    'value':experimentModels_N2.eventRecall,
    'event F1':experimentModels_N2.eventF1
})
visualTradeoff=pd.concat((auxPrecision,auxRecall))

In [212]:
fig=px.scatter(visualTradeoff,x='event F1',y='value',color='metric', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels_N2['eventF1'], y=experimentModels_N2['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

### remove outliers

In [213]:
np.setdiff1d(experimentModels_N2.test,['0006'])

array(['0001', '0002', '0003', '0005', '0007', '0009', '0010', '0011',
       '0012', '0013', '0014', '0017', '0018', '0019'], dtype=object)

In [214]:
experimentModels_N2[experimentModels_N2.test.isin(np.setdiff1d(experimentModels_N2.test,['0001','0014','0019']))][['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean().mean()

test              1.666917e+43
eventF1           8.108710e-01
eventPrecision    8.172386e-01
eventRecall       8.268100e-01
dtype: float64

In [215]:
annotationPairToGraph(gtAnnotations,processedAnnotations)